<a href="https://colab.research.google.com/github/ddah0329/SeoulPublicBike-Analysis/blob/main/rental_analysis_by_region.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive/')

# 접근하려는 폴더 설정
FOLDERNAME = 'SeoulPublicBike-Analysis'
TARGET_PATH = f'/content/drive/MyDrive/{FOLDERNAME}'

# 접근하려는 폴더로 디렉토리 이동
%cd $TARGET_PATH

# sys.path에서 다른 드라이브 경로 제거 (불필요한 접근 방지)
import sys
original_paths = sys.path.copy()
sys.path = [p for p in sys.path if p.startswith('/usr/') or p.startswith('/env/') or p == TARGET_PATH]

print("📁 현재 폴더:", TARGET_PATH)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/SeoulPublicBike-Analysis
📁 현재 폴더: /content/drive/MyDrive/SeoulPublicBike-Analysis


In [2]:
# 필요한 모듈 import하기
import os
import pandas as pd

# 1. 데이터 불러오기
file_name = 'raw_csv/bicycle_rental_ by_region(2021).csv'
df = pd.read_csv(file_name)

In [3]:
# Step 1: 불필요한 행 제거 (월별 데이터 제거, 소계만 남기기)
step1_df = df[df['월'] == '소계'].copy()
step1_df.reset_index(drop=True, inplace=True)
step1_df.head()

,년,월,계,강남구,강동구,강북구,강서구,관악구,광진구,구로구,...,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,기타
0,2017,소계,"5,031,039","128,535","79,361","31,969","116,730","108,948","303,005","83,563",...,"75,552","161,489","75,184","650,295","137,720","185,616","488,287","245,149","73,415",NaN
1,2018,소계,"10,061,684","427,050","327,554","167,753","381,050","366,488","636,660","335,908",...,"315,564","698,292","251,325","902,910","223,344","300,827","570,470","295,076","253,544",NaN
2,2019,소계,"19,074,794","814,143","674,992","339,073","1,181,327","686,181","1,208,136","688,327",...,"625,117","1,602,365","556,819","1,569,388","393,396","563,992","801,898","459,250","489,097",NaN
3,2020,소계,"23,705,176","786,869","878,510","462,869","1,992,035","814,483","1,199,629","941,328",...,"755,786","1,775,555","1,043,611","1,962,266","534,614","735,678","861,312","515,547","676,357",NaN
4,2021,소계,"32,053,367","954,181","1,273,447","576,052","3,274,017","1,030,133","1,490,398","1,219,760",...,"918,391","2,684,444","1,701,609","2,554,564","730,546","924,908","1,182,889","757,888","872,391",NaN


In [4]:
# Step 2: 숫자형 문자열을 실수형으로 변환 (쉼표 제거 → float)
step2_df = step1_df.copy()
cols_to_convert = step2_df.columns.drop(['년', '월'])

for col in cols_to_convert:
    step2_df[col] = step2_df[col].astype(str).str.replace(',', '').replace('nan', None).astype(float)

step2_df.head()

,년,월,계,강남구,강동구,강북구,강서구,관악구,광진구,구로구,...,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,기타
0,2017,소계,5031039.0,128535.0,79361.0,31969.0,116730.0,108948.0,303005.0,83563.0,...,75552.0,161489.0,75184.0,650295.0,137720.0,185616.0,488287.0,245149.0,73415.0,NaN
1,2018,소계,10061684.0,427050.0,327554.0,167753.0,381050.0,366488.0,636660.0,335908.0,...,315564.0,698292.0,251325.0,902910.0,223344.0,300827.0,570470.0,295076.0,253544.0,NaN
2,2019,소계,19074794.0,814143.0,674992.0,339073.0,1181327.0,686181.0,1208136.0,688327.0,...,625117.0,1602365.0,556819.0,1569388.0,393396.0,563992.0,801898.0,459250.0,489097.0,NaN
3,2020,소계,23705176.0,786869.0,878510.0,462869.0,1992035.0,814483.0,1199629.0,941328.0,...,755786.0,1775555.0,1043611.0,1962266.0,534614.0,735678.0,861312.0,515547.0,676357.0,NaN
4,2021,소계,32053367.0,954181.0,1273447.0,576052.0,3274017.0,1030133.0,1490398.0,1219760.0,...,918391.0,2684444.0,1701609.0,2554564.0,730546.0,924908.0,1182889.0,757888.0,872391.0,NaN


In [5]:
# Step 3: 결측치(NaN)를 0으로 대체
step3_df = step2_df.copy()
step3_df.fillna(0, inplace=True)
print(step3_df.head())

      년   월           계       강남구        강동구       강북구        강서구        관악구  \
0  2017  소계   5031039.0  128535.0    79361.0   31969.0   116730.0   108948.0   
1  2018  소계  10061684.0  427050.0   327554.0  167753.0   381050.0   366488.0   
2  2019  소계  19074794.0  814143.0   674992.0  339073.0  1181327.0   686181.0   
3  2020  소계  23705176.0  786869.0   878510.0  462869.0  1992035.0   814483.0   
4  2021  소계  32053367.0  954181.0  1273447.0  576052.0  3274017.0  1030133.0   

         광진구        구로구  ...       성북구        송파구        양천구       영등포구  \
0   303005.0    83563.0  ...   75552.0   161489.0    75184.0   650295.0   
1   636660.0   335908.0  ...  315564.0   698292.0   251325.0   902910.0   
2  1208136.0   688327.0  ...  625117.0  1602365.0   556819.0  1569388.0   
3  1199629.0   941328.0  ...  755786.0  1775555.0  1043611.0  1962266.0   
4  1490398.0  1219760.0  ...  918391.0  2684444.0  1701609.0  2554564.0   

        용산구       은평구        종로구        중구       중랑구   기타  
0  13772

In [6]:
# Step 4: 이상치 탐지 (0 이하 값)
cols_to_check = step3_df.columns.drop(['년', '월'])
outlier_report = {}

for col in cols_to_check:
    outliers = step3_df[step3_df[col] <= 0][['년', col]]
    if not outliers.empty:
        outlier_report[col] = outliers

# 이상치 출력
for region, df_out in outlier_report.items():
    print(f"\n[이상치 발견] {region}")
    print(df_out)


[이상치 발견] 기타
      년   기타
0  2017  0.0
1  2018  0.0
2  2019  0.0
3  2020  0.0
4  2021  0.0
5  2022  0.0
6  2023  0.0


In [8]:
# Step 5: '기타' 열 제거
if '기타' in step3_df.columns:
    step3_df.drop(columns='기타', inplace=True)

print(step3_df.columns)

Index(['년', '월', '계', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
       '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'],
      dtype='object')


In [13]:
# Step 6:최종 결과 출력
output_file = step3_df
print(output_file.head())

      년   월           계       강남구        강동구       강북구        강서구        관악구  \
0  2017  소계   5031039.0  128535.0    79361.0   31969.0   116730.0   108948.0   
1  2018  소계  10061684.0  427050.0   327554.0  167753.0   381050.0   366488.0   
2  2019  소계  19074794.0  814143.0   674992.0  339073.0  1181327.0   686181.0   
3  2020  소계  23705176.0  786869.0   878510.0  462869.0  1992035.0   814483.0   
4  2021  소계  32053367.0  954181.0  1273447.0  576052.0  3274017.0  1030133.0   

         광진구        구로구  ...        성동구       성북구        송파구        양천구  \
0   303005.0    83563.0  ...   378012.0   75552.0   161489.0    75184.0   
1   636660.0   335908.0  ...   556399.0  315564.0   698292.0   251325.0   
2  1208136.0   688327.0  ...   910237.0  625117.0  1602365.0   556819.0   
3  1199629.0   941328.0  ...  1055737.0  755786.0  1775555.0  1043611.0   
4  1490398.0  1219760.0  ...  1262852.0  918391.0  2684444.0  1701609.0   

        영등포구       용산구       은평구        종로구        중구       중랑구  
0 

In [16]:
# 최종 파일저장
output_file.to_csv("processed_csv/rent_summary_by_region.csv", index=False, encoding='utf-8-sig')


[해당 코드 수정할떄 사용한 GPT 프롬프트](https://chatgpt.com/share/682b6e70-7804-8003-be04-c734f69d1a7e)